# Boxcars Gem Notebook
Below you will find some sample boxcar gem code fragments.

## Env Setup
Before we get started, install the gems we need here:
```bash
gem install boxcars dotenv
```
and then create / edit .env to have SERPAPI_API_KEY and OPENAI_ACCESS_TOKEN

In [1]:
# setup for code below
require "dotenv/load"
require "boxcars"

true

## Examples
Here are several examples of using the boxcar gems. First we will start with individual Boxcars, and then we will move on to using Trains.

### Search using the Google Serp API (set SERPAPI_API_KEY in .env)

In [2]:
# showcase Google Serp search
s = Boxcars::GoogleSearch.new
s.run "what temperature is it in Phoenix?"

Question: what temperature is it in Phoenix?
Answer: High 69F


"High 69F"

### Calculator that uses ruby for hard math

In [3]:
c = Boxcars::Calculator.new
c.run "what is pi to the third power times epsilon?"

> Entering Calculator#run
what is pi to the third power times epsilon?
RubyREPL: puts(Math::PI ** 3 * Math::E)
Answer: 84.28379846823243

84.28379846823243
< Exiting Calculator#run


"84.28379846823243"

### Active Record Example using SQL code generation

In [2]:
# first stub out an in memory sqlite3 and a simple helpdesk example
require 'active_record'
require './helpdesk_sample'

-- create_table("users", {:force=>:cascade})
   -> 0.0085s
-- create_table("comments", {:force=>:cascade})
   -> 0.0006s
-- create_table("tickets", {:force=>:cascade})
   -> 0.0005s


true

In [3]:
# get the answer
boxcar = Boxcars::SQL.new
boxcar.run "how many open tickets for John?"

> Entering data#run
how many open tickets for John?
SELECT COUNT(*) FROM tickets JOIN users on tickets.user_id = users.id WHERE users.name = "John" AND tickets.status = 0;
Answer: [{"COUNT(*)"=>1}]
< Exiting data#run


"Answer: [{\"COUNT(*)\"=>1}]"

In [4]:
boxcar.run "how many comments do we have on closed tickets?"

> Entering data#run
how many comments do we have on closed tickets?
SELECT COUNT(*) FROM comments c INNER JOIN tickets t ON c.ticket_id = t.id WHERE t.status = 0;
Answer: [{"COUNT(*)"=>4}]
< Exiting data#run


"Answer: [{\"COUNT(*)\"=>4}]"

### Active Record Example using ActiveRecord code generation

In [5]:
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
helpdesk.run "how many comments do we have on open tickets?"

> Entering helpdesk#run
how many comments do we have on open tickets?
Ticket.where(status: 0).joins(:comments).count
Answer: 4
< Exiting helpdesk#run


"Answer: 4"

#### ActiveRecord is Read Only by default

In [6]:
helpdesk.run "Move all of Sally's open tickets to John"

> Entering helpdesk#run
Move all of Sally's open tickets to John
Ticket.where(user_id: User.find_by(name: "Sally").id, status: 0).update_all(user_id: User.find_by(name: "John").id)
code included destructive instruction: update_all Ticket.where(user_id: User.find_by(name: "Sally").id, status: 0).update_all(user_id: User.find_by(name: "John").id)
Error: Can not run code that makes changes in read-only mode
< Exiting helpdesk#run


"Error: Can not run code that makes changes in read-only mode"

#### But you can make changes if you want!

In [8]:
rw_helpdesk = Boxcars::ActiveRecord.new(read_only: false, name: 'Helpdesk', models: [Ticket, User, Comment])
rw_helpdesk.run "Move all of Sally's open tickets to John"

> Entering Helpdesk#run
Move all of Sally's open tickets to John
Ticket.where(user_id: User.find_by(name: "Sally"), status: 0).update_all(user_id: User.find_by(name: "John"))
Answer: 2
< Exiting Helpdesk#run


"Answer: 2"

## Putting it all together - Trains
Trains uses a series of Boxcars to compute an answer by composing answers from the union of queries to individual boxcars as needed.
### a one car train
This is similar to just running Boxcar.run

In [8]:
c = Boxcars::Calculator.new
train = Boxcars.train.new(boxcars: [c])
train.run "what is pi squared?"

> Entering Zero Shot#run
what is pi squared?
> Entering Calculator#run
pi^2
Answer: 9.869604401089358
< Exiting Calculator#run
Observation: Answer: 9.869604401089358
I now know the final answer
Final Answer: 9.869604401089358
< Exiting Zero Shot#run


"9.869604401089358"

### a two car train
Here we have a HelpDesk ActiveRecord database and a calculator at our disposal to get an answer

In [11]:
c = Boxcars::Calculator.new
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
train = Boxcars.train.new(boxcars: [c, helpdesk])
train.run "the number of open helpdesk tickets that John commented on times 2 pi?"

> Entering Zero Shot#run
the number of open helpdesk tickets that John commented on times 2 pi?
I need to find the number of open helpdesk tickets that John commented on and then multiply it by 2 pi.
Action: helpdesk
Action Input: find all open helpdesk tickets that John commented on
Observation:
5 open helpdesk tickets that John commented on
Thought:
I now need to multiply the number of helpdesk tickets by 2 pi
Action: Calculator
Action Input: 5 * 2 * pi
Observation:
31.41592653589793
Thought:
I now know the final answer
Final Answer: 31.41592653589793
< Exiting Zero Shot#run


"31.41592653589793"